In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import layers
from data import dataprep
print(tf.__version__)

In [ ]:
train_dict, test_dict = dataprep.mf_train_test()

In [ ]:
n_user = train_dict['user_id'].max() + 1
n_movie = train_dict['movie_id'].max() + 1
n_dim = 20
reg_param = 0.01
learning_rate = 0.01

In [ ]:
# create input place_holders
users = tf.placeholder(shape=[None], dtype=tf.int64)
movies = tf.placeholder(shape=[None], dtype=tf.int64)
ratings = tf.placeholder(shape=[None], dtype=tf.float32)

In [ ]:
with tf.variable_scope("embedding"):
    user_weight = tf.get_variable("user_w"
                                  , shape=[n_user, n_dim]
                                  , dtype=tf.float32
                                  , initializer=layers.xavier_initializer())
    user_bias = tf.get_variable("user_b"
                                , shape=[n_user]
                                , dtype=tf.float32
                                , initializer=tf.zeros_initializer)
    movie_weight = tf.get_variable("movie_w"
                                   , shape=[n_movie, n_dim]
                                   , dtype=tf.float32
                                   , initializer=layers.xavier_initializer())
    movie_bias = tf.get_variable("movie_b"
                                 , shape=[n_movie]
                                 , dtype=tf.float32
                                 , initializer=tf.zeros_initializer)

In [ ]:
tf.reset_default_graph()

In [ ]:
with tf.name_scope("inference"):
    user_emb = tf.nn.embedding_lookup(user_weight, users)
    u_b = tf.nn.embedding_lookup(user_bias, users)
    movie_emb = tf.nn.embedding_lookup(movie_weight, movies)
    m_b = tf.nn.embedding_lookup(movie_bias, movies)
    pred = tf.reduce_sum(tf.multiply(user_emb, movie_emb), 1) + u_b + m_b

In [ ]:
with tf.name_scope("loss"):
    reg_loss = layers.apply_regularization(layers.l2_regularizer(scale=reg_param),
                                           weights_list=[user_weight, movie_weight])
    loss = tf.nn.l2_loss(pred - ratings) + reg_loss
    train_ops = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    rmse = tf.sqrt(tf.reduce_mean(tf.pow(pred - ratings, 2)))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_input_dict = {users: train_dict['user_id']
        , movies: train_dict['movie_id']
        , ratings: train_dict['rating']}
    test_input_dict = {users: test_dict['user_id']
        , movies: test_dict['movie_id']
        , ratings: test_dict['rating']}
    for i in range(1000):
        sess.run([train_ops], feed_dict=train_input_dict)
        if i % 10 == 0:
            rmse_train = sess.run(rmse, feed_dict=train_input_dict)
            rmse_test = sess.run(rmse, feed_dict=test_input_dict)
            print("train rmse: %.3f , test rmse %.3f" % (rmse_train, rmse_test))